EMBEDDING CREATION

In [2]:
text_chunks = ["my name is tarique nomani", "I am a data scientist", "I have 3 years of expereicne", "I have cloud certifications"]

In [3]:
from sentence_transformers import SentenceTransformer

d:\RAG2.o\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = SentenceTransformer("thenlper/gte-large")

In [5]:
embeddings = model.encode(text_chunks, normalize_embeddings=True).tolist()

In [14]:
type(embeddings), len(embeddings), len(embeddings[0])
# type is list
# there is 4 sentences in the list
# each sentence have 1024 embedding dimension

(list, 4, 1024)

In [15]:
data = [embeddings, text_chunks]

In [24]:
type(data), data[-1], data[-2][0][:5]
# third one is showing the embedding vector of the first index and out of 1024 dimension for each index its only showing first 5 vector

(list,
 ['my name is tarique nomani',
  'I am a data scientist',
  'I have 3 years of expereicne',
  'I have cloud certifications'],
 [-0.023396523669362068,
  -0.0034130122512578964,
  -0.03790540248155594,
  -0.019632985815405846,
  -0.016310932114720345])

In [28]:
!pip install -qU "langchain-chroma>=0.1.2"

CREATE A MILVUS CLIENT AND PUSH THE VECTOR EMBEDDING TO ITS DATABASE

In [29]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [30]:
vector_store.add_documents(documents=text_chunks)

AttributeError: 'str' object has no attribute 'id'

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

In [31]:
test_data = "I had chocolate chip pancakes and scrambled eggs for breakfast this morning. The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. Building an exciting new project with LangChain - come check it out! Robbers broke into the city bank and stole $1 million in cash. Wow! That was an amazing movie. I can't wait to see it again. Is the new iPhone worth the price? Read this review to find out. The top 10 soccer players in the world right now. LangGraph is the best framework for building stateful, agentic applications! The stock market is down 500 points today due to fears of a recession. I have a bad feeling I am going to get deleted :("

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [34]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=30,
    chunk_overlap=5,
    length_function=len,
    is_separator_regex=False,
)

In [35]:
texts = text_splitter.create_documents([test_data])
print(texts[0])
print(texts[1])

page_content='I had chocolate chip pancakes'
page_content='and scrambled eggs for'


In [38]:
print(type(texts))
for text in texts:
    print(text)

<class 'list'>
page_content='I had chocolate chip pancakes'
page_content='and scrambled eggs for'
page_content='for breakfast this morning.'
page_content='The weather forecast for'
page_content='for tomorrow is cloudy and'
page_content='and overcast, with a high of'
page_content='of 62 degrees. Building an'
page_content='an exciting new project with'
page_content='with LangChain - come check'
page_content='it out! Robbers broke into'
page_content='into the city bank and stole'
page_content='$1 million in cash. Wow! That'
page_content='That was an amazing movie. I'
page_content='I can't wait to see it again.'
page_content='Is the new iPhone worth the'
page_content='the price? Read this review'
page_content='to find out. The top 10'
page_content='10 soccer players in the'
page_content='the world right now.'
page_content='now. LangGraph is the best'
page_content='best framework for building'
page_content='stateful, agentic'
page_content='applications! The stock'
page_content='market is do

In [36]:
text_splitter.split_text(test_data)[:2]

['I had chocolate chip pancakes', 'and scrambled eggs for']

In [45]:
from pymilvus import connections

try:
    connections.connect(host="127.0.0.1", port=19530)
    #connections.connect(host="host.docker.internal", port=19530)
    print("Connected to Milvus successfully!")
except Exception as e:
    print(f"Connection failed: {e}")


Connected to Milvus successfully!
